# Introduction

This script provides a single, consolidated experimental framework for simulating three social choice algorithms: Plurality, Veto, and Borda.

By combining all functions into one file and using a central execution block, it eliminates code redundancy and provides a robust tool for comparing the performance of different voting rules. 

The script takes user input to select
the desired voting rule and then outputs key metrics, including distortion.

# Import Necessary Libralies

In [2]:
import random
import numpy as np
import collections

In [ ]:
# PART 1: Data Generation Functions
def generate_utilities(num_agents, num_alternatives, distribution_type='uniform'):
    """Generates a 2D array of cardinal utility values based on a specified distribution."""
    utilities = []
    valid_distributions = ['uniform', 'normal', 'strong_preference', 'unit_sum']
    if distribution_type not in valid_distributions:
        print(f"Warning: Invalid distribution type '{distribution_type}'. Defaulting to 'uniform'.")
        distribution_type = 'uniform'
        
    for _ in range(num_agents):
        agent_utilities = []
        if distribution_type == 'uniform':
            agent_utilities = [random.uniform(0, 1) for _ in range(num_alternatives)]
        elif distribution_type == 'normal':
            agent_utilities = [np.random.normal(loc=0.5, scale=0.2) for _ in range(num_alternatives)]
            agent_utilities = [max(0, min(1, u)) for u in agent_utilities]
        elif distribution_type == 'strong_preference':
            agent_utilities = [0.00] * num_alternatives
            strong_choice_index = random.randint(0, num_alternatives - 1)
            agent_utilities[strong_choice_index] = 1.00
        elif distribution_type == 'unit_sum':
            points = sorted([0.0] + [random.uniform(0, 1) for _ in range(num_alternatives - 1)] + [1.0])
            agent_utilities = [points[i+1] - points[i] for i in range(num_alternatives)]
            
        utilities.append(agent_utilities)
    return utilities


def generate_preferences(utilities):
    """
    Generates a list of agent preferences based on their cardinal utilities.
    Each agent's preference list is sorted from their most-preferred alternative
    (highest utility) to their least-preferred (lowest utility).
    """
    preferences = []
    num_alternatives = len(utilities[0])
    alternative_names = [f'A{i + 1}' for i in range(num_alternatives)]

    # Iterate through each agent's utility list
    for agent_utilities in utilities:
        
        # Pair each alternative name with its utility for this agent
        utility_pairs = list(zip(alternative_names, agent_utilities))

        # Sort the pairs in descending order based on utility (the 2nd item in the pair)
        utility_pairs.sort(key=lambda pair: pair[1], reverse=True)

        # Extract just the alternative names from the sorted pairs to create the ballot
        rational_preference_list = [alt[0] for alt in utility_pairs]
        
        preferences.append(rational_preference_list)
        
    return preferences

# PART 2: Voting Algorithm Functions (Specific)
def plurality_voting(preferences):
    """Implements the Plurality voting rule to find the winning alternative."""
    if not preferences: return {"winner": None, "score": 0, "scores": {}}
    vote_counts = collections.Counter(pref[0] for pref in preferences)
    winner, max_votes = max(vote_counts.items(), key=lambda item: item[1])
    return {"winner": winner, "score": max_votes, "scores": dict(vote_counts)}

def veto_voting(preferences):
    """Implements the Veto voting rule to find the winning alternative."""
    if not preferences or not preferences[0]: return {"winner": None, "score": 0, "scores": {}}
    num_alternatives = len(preferences[0])
    veto_scores = {alt: 0 for alt in preferences[0]}
    for pref_list in preferences:
        veto_scores[pref_list[-1]] += 1
    winner, min_vetoes = min(veto_scores.items(), key=lambda item: item[1])
    scores_for_display = {alt: len(preferences) - score for alt, score in veto_scores.items()}
    return {"winner": winner, "score": len(preferences) - min_vetoes, "scores": scores_for_display}

def borda_voting(preferences):
    """Implements the Borda voting rule to find the winning alternative."""
    if not preferences or not preferences[0]: return {"winner": None, "score": 0, "scores": {}}
    num_alternatives = len(preferences[0])
    borda_scores = collections.defaultdict(int)
    for pref_list in preferences:
        for i, alternative in enumerate(pref_list):
            borda_scores[alternative] += (num_alternatives - 1 - i)
    winner, max_score = max(borda_scores.items(), key=lambda item: item[1])
    return {"winner": winner, "score": max_score, "scores": dict(borda_scores)}


# PART 3: Distortion Calculation Function (Shared)
def calculate_distortion(utilities, winner):
    """Calculates the distortion of a voting rule's outcome."""
    if not utilities:
        return { "optimal_alternative": None, "optimal_social_welfare": 0,
            "chosen_alternative": winner, "achieved_social_welfare": 0,
            "distortion": 0}
    num_agents = len(utilities)
    num_alternatives = len(utilities[0])
    alternative_names = [f'A{i + 1}' for i in range(num_alternatives)]
    alt_to_index = {name: i for i, name in enumerate(alternative_names)}
    social_welfare = [sum(utilities[i][j] for i in range(num_agents)) for j in range(num_alternatives)]
    optimal_social_welfare = max(social_welfare)
    optimal_alternative = alternative_names[social_welfare.index(optimal_social_welfare)]
    achieved_social_welfare = social_welfare[alt_to_index[winner]] if winner in alt_to_index else 0
    distortion = optimal_social_welfare / achieved_social_welfare if achieved_social_welfare > 0 else float('inf')
    return {
        "optimal_alternative": optimal_alternative,
        "optimal_social_welfare": optimal_social_welfare,
        "chosen_alternative": winner,
        "achieved_social_welfare": achieved_social_welfare,
        "distortion": distortion}

# Main Experimental Framework

if __name__ == "__main__":
    # User Input Panel
    voting_rule = str.lower(input("Select voting rule ('plurality', 'veto', or 'borda'): "))
    num_agents = int(input("Enter the number of agents: "))
    num_alternatives = int(input("Enter the number of alternatives: "))
    distribution_type = str.lower(input("Select distribution ('uniform', 'normal', 'strong_preference', or 'unit_sum'): "))

    if voting_rule not in ['plurality', 'veto', 'borda']:
        print("Error: Invalid voting rule. Please choose 'plurality', 'veto', or 'borda'.")
    else:
        
        # 1. Generate the cardinal utilities (The "Truth")
        cardinal_utilities = generate_utilities(num_agents, num_alternatives, distribution_type)
        
        # 2. Generate the preferences (The "Ballot")
        ordinal_preferences = generate_preferences(cardinal_utilities)

        if voting_rule == 'plurality':
            results = plurality_voting(ordinal_preferences)
        elif voting_rule == 'veto':
            results = veto_voting(ordinal_preferences)
        else: # 'borda'
            results = borda_voting(ordinal_preferences)

        winner = results['winner']
        
        # 3. Calculate distortion
        distortion_results = calculate_distortion(cardinal_utilities, winner)
        
        # Analysis Output Panel
        print(f"\n=== Analysis Results for {voting_rule.capitalize()} Voting ===")
        print(f"  - Parameters:")
        print(f"    - Number of Agents: {num_agents}")
        print(f"    - Number of Alternatives: {num_alternatives}")
        print(f"    - Distribution Type: {distribution_type}")
        
        print("\n=== Analysis Results ===")
        print(f"  - Optimal Alternative: {distortion_results['optimal_alternative']}")
        print(f"  - Optimal Social Welfare: {distortion_results['optimal_social_welfare']:.2f}")
        print(f"  - Chosen Alternative (by {voting_rule.capitalize()}): {distortion_results['chosen_alternative']}")
        print(f"  - Achieved Social Welfare: {distortion_results['achieved_social_welfare']:.2f}")
        print(f"  - Distortion: {distortion_results['distortion']:.2f}")



=== Analysis Results for Plurality Voting ===
  - Parameters:
    - Number of Agents: 10
    - Number of Alternatives: 5
    - Distribution Type: normal

=== Analysis Results ===
  - Optimal Alternative: A1
  - Optimal Social Welfare: 4.64
  - Chosen Alternative (by Plurality): A5
  - Achieved Social Welfare: 4.26
  - Distortion: 1.09
